In [1]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import yfinance as yf
import numpy as np

c:\Users\Daksh Vashist\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
# Methodology reference: https://www.msci.com/eqb/methodology/meth_docs/MSCI_Momentum_Indexes_Methodology_Aug2021.pdf

sp500_constituents = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_constituents.drop(sp500_constituents.columns[[1, 2, 3, 4, 5, 6, 7]], axis=1, inplace=True)

In [3]:
sp500_list = sp500_constituents['Symbol'].to_list()

sp500_list.remove('BRK.B')
sp500_list.remove('BF.B')

In [7]:
final_df = pd.DataFrame({'Stock Code': sp500_list,
                             'Momentum Score (C)': np.nan})

In [5]:
start = dt.datetime(2019, 1, 1)
end = dt.datetime(2023, 12, 31)

In [6]:
# Creating an empty dataframe for all of the stocks of S&P 500
sp500_histdata = pd.DataFrame({
    'Date':pd.date_range(start, end)
    }
)
#sp500_histdata[sp500_list] = np.nan

In [ ]:
sp500_histdata.tail()

In [ ]:
# Data extraction using STOOQ

name = 'AMZN'
df = web.DataReader(name, 'stooq', start, end)
df.rename(columns={'Close':name}, inplace=True)
df.reset_index(inplace=True)
#df.drop(df.columns[[1, 2, 3, 5]], axis=1, inplace=True)

df.head()

In [9]:
# Data extraction using YFINANCE

amazon = yf.Ticker('AMZN')
amazon_data = amazon.history(period='5y')
amazon_data.reset_index(inplace=True)
amazon_data.drop(amazon_data.columns[[1, 2, 3, 5, 6, 7]], axis=1, inplace=True)

In [10]:
 # Download stock close prices of a list of stocks using yfinance

for stock in sp500_list:
    try:
        df = yf.Ticker(stock).history(period='5y')
        df.rename(columns={'Close':stock}, inplace=True)
        df.reset_index(inplace=True)
        df.drop(df.columns[[1, 2, 3, 5, 6, 7]], axis=1, inplace=True)
        df.to_csv(stock + '.csv')
    except:
        pass

In [29]:
# Changing format of Date column to datetime type

for stock in sp500_list:
    try:
        df = pd.read_csv(stock+'.csv')
        df['Date'] = pd.to_datetime(df['Date'], utc=True)
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        df.to_csv(stock+'.csv')
    except:
        print ('error')


In [83]:
# Historical data prepared by joining stock price data of all S&P 500 stocks

for stock in sp500_list:
    df = pd.read_csv(stock + '.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df.drop(df.columns[[0, 1]], axis=1, inplace=True)
    sp500_histdata = pd.merge(left=sp500_histdata,
                     right=df,
                     left_on="Date",
                     right_on="Date",
                     how="outer")

sp500_histdata.to_csv('sp500_histdata.csv')
    

In [85]:
df = pd.read_csv('sp500_histdata.csv')
df.drop(df.columns[[0]], axis=1, inplace=True)
df.index = df['Date']
df.drop(df.columns[[0]], axis=1, inplace=True)
df.dropna(axis=0, how='all', inplace=True)

df.to_csv('sp500_histdata.csv')

In [ ]:
df = pd.read_csv('sp500_histdata.csv')
df.index = df['Date']
df.drop(df.columns[[0]], axis=1, inplace=True)
df.index = pd.to_datetime(df.index)
df.head()

In [ ]:
'''
Renaming pandas columns

for stock in sp500_list:
    df.rename(columns={stock+'_x':stock}, inplace=True)

'''

In [126]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.resample('M', on='Date').last()

In [ ]:
df = pd.read_csv('sp500_histdata.csv')
df.index = df['Date']
df.drop(df.columns[[0]], axis=1, inplace=True)

In [ ]:
### Calculating 6 month and 12 month returns

#df['MMM_t-6'] = df['MMM'].shift(6)
#df['MMM_t-12'] = df['MMM'].shift(12)

In [150]:
# Adding 6m and 12m shift column to further calculate returns for the period

for stock in sp500_list:
    df = pd.read_csv(stock+'.csv')
    df[stock+'_6m'] = df[stock].shift(6)
    df[stock+'_12m'] = df[stock].shift(12)
    df.to_csv(stock+'.csv')

print('lets gooo!')

lets gooo!


In [6]:
# 6m and 12m returns minus 3 month us t bill rate = 5.22% as of 5th feb 2024

# Need to deduct t-bill return

for stock in sp500_list:
    df = pd.read_csv(stock+'.csv')
    df[stock+'_6m_ret'] = (( df[stock]/df[stock+'_6m'] ) - 1 )
    df[stock+'_12m_ret'] = (( df[stock]/df[stock+'_12m'] ) - 1 )
    df.to_csv(stock+'.csv')

In [36]:
final_df = pd.read_csv('final_df.csv')
final_df[['Price_t_1', 'Price_t_7', 'Price_t_13']] = None
final_df.to_csv('final_df.csv', index=False)

final_df.head()

,Stock Code,Price_t_1,Price_t_7,Price_t_13
0,MMM,None,None,None
1,AOS,None,None,None
2,ABT,None,None,None
3,ABBV,None,None,None
4,ACN,None,None,None


In [4]:
## Rebelancing date assumed to be 1st January 2024

t_1 = '2023-11-30'
t_7 = '2023-05-31'
t_13 = '2022-05-31'

In [ ]:
# Rebalance date: 1st January 2024. t-1 = 2023-11-30, t-7 = 2023-05-31, t-13 = 2022-05-31


### Adding values for t_1 into final_df

final_df = pd.read_csv('final_df.csv')

for item in range(0,501):
    df = pd.read_csv(final_df['Stock Code'][item] + '.csv')
    price = df.loc[df['Date']==t_1][final_df['Stock Code'][item]]
    final_df['Price_t_1'][item] = price

final_df.to_csv('final_df.csv', index=False)

In [ ]:
### Adding values for t_7 into final_df

final_df = pd.read_csv('final_df.csv')

for item in range(0,501):
    df = pd.read_csv(final_df['Stock Code'][item] + '.csv')
    price = df.loc[df['Date']==t_7][final_df['Stock Code'][item]]
    final_df['Price_t_7'][item] = price
    final_df.to_csv('final_df.csv', index=False)

In [ ]:
### Adding values for t_13 into final_df

final_df = pd.read_csv('final_df.csv')

for item in range(0,501):
    df = pd.read_csv(final_df['Stock Code'][item] + '.csv')
    price = df.loc[df['Date']==t_13][final_df['Stock Code'][item]]
    final_df['Price_t_13'][item] = price
    final_df.to_csv('final_df.csv', index=False)

In [6]:
final_df = pd.read_csv('final_df.csv')
print(final_df.head())

  Stock Code   Price_t_1   Price_t_7  Price_t_13
0        MMM   99.070000   90.510773  137.736343
1        AOS   75.064651   63.130753   58.191288
2        ABT  103.788605  100.468941  113.559708
3       ABBV  141.042435  133.816589  137.606705
4        ACN  331.927948  302.450317  290.394440


In [7]:
### Creating columns for 6month and 12month Price momentum score (unstandardized)

final_df['6m_mom'] = (final_df['Price_t_1']/final_df['Price_t_7']) - 1
final_df['12m_mom'] = (final_df['Price_t_1']/final_df['Price_t_13']) - 1
final_df.head()

,Stock Code,Price_t_1,Price_t_7,Price_t_13,6m_mom,12m_mom
0,MMM,99.070000,90.510773,137.736343,0.094566,-0.280727
1,AOS,75.064651,63.130753,58.191288,0.189035,0.289964
2,ABT,103.788605,100.468941,113.559708,0.033042,-0.086044
3,ABBV,141.042435,133.816589,137.606705,0.053998,0.024968
4,ACN,331.927948,302.450317,290.394440,0.097463,0.143024


In [8]:
# creating column for risk adjusted scores: 6m and 12m

final_df[['6m_risk_adj', '12m_risk_adj']] = None

In [ ]:
## Calculating standard deviation --> Annualized Standard Deviation of weekly local price returns over the period of 3 years 
## 3 years = 1st January 2021 - 31st December 2023
import statistics
y_bgn = '2021-01-04'
y_end = '2023-12-29'


final_df = pd.read_csv('final_df.csv')

for item in range(0,501):
    df = pd.read_csv(final_df['Stock Code'][item] + '.csv')
    final_df['6m_risk_adj'][item] = statistics.stdev([df])
    final_df.to_csv('final_df.csv', index=False)